## Создание дашборда для опредения уровня NPS

In [1]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
# подключаем базу
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
# Выполняем sql запрос 
query = """
           SELECT u.user_id AS id,
                  u.lt_day AS days_of_life,
                  CASE
                      WHEN u.lt_day <= 365  THEN 'TRUE'
                      ELSE 'FALSE'
                  END AS is_new,
                  u.age,
                  CASE
                      WHEN gender_segment == 1 THEN 'female'
                      WHEN gender_segment == 0 THEN 'male'
                      ELSE 'n/a'
                  END AS gender_segment,
                  u.os_name AS os_type,
                  u.cpe_type_name device_type,
                  loc.country,
                  loc.city,
                  SUBSTR(ag.title, 3, 6) AS age_segment,
                  SUBSTR(tr.title, 3, 6) AS traffic_segment,
                  SUBSTR(lt.title, 3, 6) AS lifetime_segment,
                  u.nps_score, 
                  CASE
                      WHEN u.nps_score BETWEEN 0 AND 6 THEN 'detractors'
                      WHEN u.nps_score BETWEEN 7 AND 8 THEN 'passives'
                      WHEN u.nps_score BETWEEN 9 AND 10 THEN 'promoters'
                  END AS nps_group
           FROM user AS u
           LEFT OUTER JOIN location AS loc ON u.location_id = loc.location_id
           LEFT OUTER JOIN age_segment AS ag ON u.age_gr_id = ag.age_gr_id
           LEFT OUTER JOIN traffic_segment AS tr ON u.tr_gr_id = tr.tr_gr_id
           LEFT OUTER JOIN lifetime_segment AS lt ON u.lt_gr_id = lt.lt_gr_id;
        """

In [4]:
# получаем данные и проверяем корректность
df = pd.read_sql(query, engine)
df.head(5)

,id,days_of_life,is_new,age,gender_segment,os_type,device_type,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,FALSE,45.0,female,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,promoters
1,A001WF,2344,FALSE,53.0,male,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,promoters
2,A003Q7,467,FALSE,57.0,male,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,promoters
3,A004TB,4190,FALSE,44.0,female,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,promoters
4,A004XT,1163,FALSE,24.0,male,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,promoters


In [5]:
# получаем готовую витрину 
df.to_csv('telecomm_csi_tableau.csv', index=False)

In [6]:
 # Просмотрим общие данные
df.describe()

,days_of_life,age,nps_score
count,502493.000000,501939.000000,502493.000000
mean,1868.841439,39.621946,7.508562
std,1683.701762,11.188249,3.020378
min,-21.000000,10.000000,1.000000
25%,533.000000,31.000000,5.000000
50%,1239.000000,38.000000,9.000000
75%,3064.000000,47.000000,10.000000
max,9162.000000,89.000000,10.000000


В стобце с "днями жизни" можем заметить отрицательные значения, необходимо проверить % данных и решить , что делать с данными.

In [7]:
# уточним кол-во, тип данных и т.д
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                502493 non-null  object 
 1   days_of_life      502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    502493 non-null  object 
 5   os_type           502493 non-null  object 
 6   device_type       502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


Есть потери в столбце с возрастом (age), но так так потери незначительны данные корректировать не будем.

In [8]:
# просматриваем данные с минусовыми значениями и равными нулю
df[df['days_of_life'] <= 0]

,id,days_of_life,is_new,age,gender_segment,os_type,device_type,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
7666,AEAC2R,-8,TRUE,NaN,n/a,ANDROID,SMARTPHONE,Россия,Томск,n/a,15-20,36+,1,detractors
53997,CS0HF8,-2,TRUE,NaN,n/a,ANDROID,SMARTPHONE,Россия,Ижевск,n/a,20-25,36+,10,promoters
71577,DORPT2,-4,TRUE,NaN,n/a,ANDROID,SMARTPHONE,Россия,Москва,n/a,20-25,36+,4,detractors
100152,F5O3CG,-21,TRUE,NaN,n/a,ANDROID,SMARTPHONE,Россия,Чита,n/a,35-40,36+,7,passives
103223,FBC993,0,TRUE,NaN,n/a,ANDROID,SMARTPHONE,Россия,Москва,n/a,1-5,1,5,detractors
228087,LSE939,-13,TRUE,NaN,n/a,ANDROID,SMARTPHONE,Россия,Краснодар,n/a,15-20,36+,1,detractors
284966,OQO5GZ,0,TRUE,NaN,male,ANDROID,SMARTPHONE,Россия,Томск,n/a,10-15,1,8,passives
325212,QTT7IR,-13,TRUE,NaN,n/a,IOS,SMARTPHONE,Россия,Москва,n/a,10-15,36+,7,passives
347784,RZUS6H,-6,TRUE,NaN,n/a,ANDROID,SMARTPHONE,Россия,Красноярск,n/a,30-35,36+,3,detractors
385897,TYWQW4,-12,TRUE,NaN,n/a,IOS,SMARTPHONE,Россия,Новосибирск,n/a,1-5,36+,5,detractors


In [9]:
# просматриваем данные, где не указан пол 
df.query('gender_segment  in ("n/a")').count()

id                  1301
days_of_life        1301
is_new              1301
age                  787
gender_segment      1301
os_type             1301
device_type         1301
country             1301
city                1301
age_segment         1301
traffic_segment     1301
lifetime_segment    1301
nps_score           1301
nps_group           1301
dtype: int64

1) Получаем список "отрицательных" пользователей, что можем заметить:

 * Помимо отрицательных, есть нулевые значения;
 * У пользователей не указан возраст;
 * Относятся к сегменту к возрастному сегменту n/a, возможно, если возраст не указан по дефолту пользователь помечается в данный сегмент;
 * Все пользователи распознаются, как новые;
 * Пользователи с отрицательными значениями помечаются, как - 36+ в лайфтайм, однако пользователи с 0, как 1.
 
2) Так же отметим, что у некоторых пользователей не указан пол. 

Т.к у нас учебный проект и потери незначительны (<5%), то мы не будем корректировать, либо удалять данные, однако стоило бы заметить и обсудить это с коллегами на предмет ошибок и искажений данных в дальнейшем.

### Ссылки на дашборд на сайте Tableau Public и pdf-file c презентацией:

- https://public.tableau.com/app/profile/tim21051960/viz/telecomm_csi_update/Dashboard1?publish=yes - Дашборд
- https://disk.yandex.ru/i/N84JydCFqe3BKA - Презентация